In [0]:
dbutils.fs.ls("/Volumes/big_data_ii_2025/spark_examples/spark_data/people/people.csv")

# Lectura CSVs

En este notebook simulamos la lectura de archivos CSV del dataset de _wine quality_ utilizando Structured Streaming.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType


wine_schema = StructType([
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True)   
])

streamFiles = (spark.readStream.format("cloudFiles")
              .option("cloudFiles.format","csv")
              .option("header","false")
              .option("sep",",")
              .option("cloudFiles.schemaLocation", 
                      "/Volumes/big_data_ii_2025/spark_examples/schema_people")
              .load("/Volumes/big_data_ii_2025/spark_examples/spark_data/people/"))

#Nota: Se le da un directorio, no un archivo, y se pueden agregar archivos posteriormente

In [0]:
dbutils.fs.ls("/Volumes/big_data_ii_2025/spark_examples/schema_people")

El siguiente paso es definir adonde lo vamos a guardar - en este caso creamos una nueva tabla _stream\_people_.

NOTA: Structured Streaming normalmente se utiliza para ingestar datos a la capa bronze del data lake/lakehouse, con un procesamiento muy liviano.

In [0]:
q = (streamFiles.writeStream
    .option(
    "checkpointLocation","/Volumes/big_data_ii_2025/spark_examples/chpt_people")
    .trigger(availableNow=True)
    .outputMode("append") 
    .table("big_data_ii_2025.spark_examples.stream_people"))

#NOTA: Table retorna un streaming query, no es necesario utilizar la función `start()`

In [0]:
#Revisamos el estado:
print(q.status)

In [0]:
q.stop()

In [0]:
%sql
SELECT * FROM big_data_ii_2025.spark_examples.stream_people